<a href="https://colab.research.google.com/github/AnuBolishetty/Anubolishetty_INFO5731_Spring2026/blob/main/In_class_exercises_4_Text_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [8]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [10]:
# Q1 — ANSWER CELL
FILE_PATH = "/content/support_messages.txt"
import pandas as pd
# TODO: load the TAB-separated file into df
# Hint: pd.read_csv(FILE_PATH, sep="\t")
df = pd.read_csv(FILE_PATH, sep="\t", names=["id", "message"])

# TODO: print df.shape and df.head(3)
print("Shape:", df.shape)
print("\nFirst 3 rows:")
print(df.head(3))


Shape: (9, 2)

First 3 rows:
   id                                            message
0  id                                            message
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...
2   2  Refund please!!! I was charged 2 times... invo...


## Q2 (3 points) — Answer below

In [11]:
# Q2 — ANSWER CELL
# TODO: create word_count, char_count, num_count, upper_word_count
# Hint for digits: df["message"].str.count(r"\d")
df["word_count"] = df["message"].str.split().apply(len)
df["char_count"] = df["message"].str.len()
df["num_count"] = df["message"].str.count(r"\d")
# Hint for ALL-CAPS words: count tokens where token.isupper()
def count_upper_words(text):
    tokens = str(text).split()
    return sum(1 for word in tokens if word.isupper())

df["upper_word_count"] = df["message"].apply(count_upper_words)

# TODO: display/print the full DataFrame
print(df)


   id                                            message  word_count  \
0  id                                            message           1   
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...          12   
2   2  Refund please!!! I was charged 2 times... invo...          11   
3   3        Great service, thanks! arrived in 2 days :)           8   
4   4  WHY is my package DAMAGED??? tracking says del...           8   
5   5  Need to change address: 7421 Frankford Rd Apt ...          12   
6   6  Support ticket: ORD-1050. Call me at (469) 555...           9   
7   7  I can’t login— password reset link not working...          10   
8   8  Item missing from box. pls send replacement!! ...          10   

   char_count  num_count  upper_word_count  
0           7          0                 0  
1          73          4                 2  
2          71          9                 3  
3          43          1                 0  
4          55          0                 2  
5        

## Q3 (3 points) — Answer below

In [14]:
# Q3 — ANSWER CELL
# Option A (sklearn stopwords):
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
STOPWORDS = set(ENGLISH_STOP_WORDS)
import re

# Option B (nltk stopwords):
# import nltk
# nltk.download("stopwords")
# from nltk.corpus import stopwords
# STOPWORDS = set(stopwords.words("english"))

def clean_text(text: str) -> str:
    # Ensure text is a string
    text = str(text)
    # 1) lowercase
    text = text.lower()
    # 2) remove punctuation/symbols (keep letters/numbers/spaces)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    # 3) remove English stopwords
    text = " ".join([word for word in text.split() if word not in STOPWORDS])
    # 4) remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# TODO: create df["clean"] using clean_text
df["clean"] = df["message"].apply(clean_text)

# TODO: print original and clean for id=1 and id=4
print("Original and Cleaned Messages for id=1 and id=4:")
for idx in [1, 4]:
    # Adjusted to account for the header row issue from Q1, assuming 'id' column contains actual IDs
    # Find rows where the 'id' column (which is currently a string) matches the desired integer ID
    original_message = df.loc[df['id'].astype(str) == str(idx), 'message'].iloc[0]
    clean_message = df.loc[df['id'].astype(str) == str(idx), 'clean'].iloc[0]
    print(f"\nID: {idx}")
    print(f"  Original: {original_message}")
    print(f"  Clean:    {clean_message}")

Original and Cleaned Messages for id=1 and id=4:

ID: 1
  Original: Hi!! My ORDER is late :(  Order# ORD-1042. Email me at sara.Ali@gmail.com
  Clean:    hi order late order ord1042 email saraaligmailcom

ID: 4
  Original: WHY is my package DAMAGED??? tracking says delivered...
  Clean:    package damaged tracking says delivered


## Q4 (2 points) — Answer below

In [12]:
# Q4 — ANSWER CELL
# order_id pattern: r"ORD-\d{4}" with re.IGNORECASE
df["order_id"] = df["message"].str.extract(r"(ORD-\d{4})", flags=re.IGNORECASE)

# email pattern (simple): r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"
# TODO: create df["order_id"] and df["email"]
df["email"] = df["message"].str.extract(
    r"([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})"
)
# TODO: print/display df[["id", "order_id", "email"]]
print(df[["id", "order_id", "email"]])

   id  order_id                  email
0  id       NaN                    NaN
1   1  ORD-1042     sara.Ali@gmail.com
2   2  ORD-1042                    NaN
3   3       NaN                    NaN
4   4       NaN                    NaN
5   5       NaN                    NaN
6   6  ORD-1050                    NaN
7   7       NaN  mehri.sattari@unt.edu
8   8  ord-1060                    NaN


## Q5 (1 point) — Answer below

In [15]:
# Q5 — ANSWER CELL
# Hint: from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# 1) fit TF-IDF on df["clean"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean"])

# 2) compute average TF-IDF per term across documents
avg_tfidf = np.mean(X.toarray(), axis=0)
terms = vectorizer.get_feature_names_out()
# 3) print top 5 terms + their average scores
tfidf_df = pd.DataFrame({
    "term": terms,
    "avg_tfidf": avg_tfidf
})
top5 = tfidf_df.sort_values(by="avg_tfidf", ascending=False).head(5)
# TODO
print("Top 5 Keywords by Average TF-IDF:")
print(top5.round(4))


Top 5 Keywords by Average TF-IDF:
       term  avg_tfidf
27  message     0.1111
34    order     0.1014
30  ord1042     0.0740
16    email     0.0710
14     days     0.0497


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
